In [151]:
# makes a ticket
def ticket_generator(t, o, a, l):
    ticket={
        'time': t,
        'owner': o,
        'action': a,
        'level': l
    }
    return ticket;

#test 1
t=ticket_generator(1, 1, 1, 1)
print(t)



{'time': 1, 'owner': 1, 'action': 1, 'level': 1}


In [152]:
import math
import random

def nextTime(rateParameter):
    return -math.log(1.0 - random.random()) / rateParameter

In [153]:
def network_writer(n):
    connect=[0]*n
    for i in range(0, n):
        a=list(range(n))
        a.remove(i)
        connect[i]=a
    return connect;

#test
print(network_writer(5))
            

[[1, 2, 3, 4], [0, 2, 3, 4], [0, 1, 3, 4], [0, 1, 2, 4], [0, 1, 2, 3]]


In [154]:
spreadparam=.1
cureparam=.05
curetimearray=[];
timeline=[]
def spreadtickethandler(tic, timeline, spreadparam, cureparam, ppl_state, t):
    ow=tic['owner']
#1
    if(ppl_state[ow]!=1):
        print('Error -- individual {:d} is not infected, but was passed to spreadtickethandler!' .format(ow))
        #break #is there a printf / how do you raise an error?  #SOMEHOW RAISE AND ERROR AND ESC
    else:
#2
        targ=tic['action'] #this action should not be -1 bc this is passed to spread ticket handler
        if(ppl_state[targ]==1):
            print('{:d} is already infected!' .format(targ)) #targ=targ #just to make python shut up NOT REAL CODE
        else:
            ppl_state[targ]=1
            t=tic['time']
            ct=t+nextTime(cureparam)
            curetimearray.append(ct) #curetime array -- purpose??
            st=t+nextTime(spreadparam)
            l=1
            if(st<ct):
                a=random.choice(connect[targ])
                l=1 #DO WE NEED LEVELS AT ALL?
                #FAILED TO NAME OR INSERT INTO TIMELINE
                tinf=ticket_generator(st, targ, a, l)
                add_to_timeline(tinf, timeline) #???
                tcure=ticket_generator(ct, targ, -1, l) #cure ticket -- do we gen here?
                add_to_timeline(tcure, timeline) #??
            else: 
                tcure=ticket_generator(ct, targ, -1, l) #cure ticket
                add_to_timeline(tcure, timeline) #??
#3
        #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
#4
        st2=t+nextTime(spreadparam)
        i=timeholder
        curefound=0
        while curefound==0:
            tim=timeline[i]
            if(tim['owner']==ow):
                if(tim['action']==-1):
                    ct_ow=tim['time']
                    curefound=1
            i=i+1
            
        if(st2<ct_ow):
            spread2tic=ticket_generator(st2, ow, targ, 1)
            add_to_timeline(spread2tic, timeline)
        
        #return infected_count;
        return [timeline, ppl_state];
            
        
    
        
def curetickethandler(tic, ppl_state):
    ow=tic['owner']
    ppl_state[ow]=0
    #infected_count=sum(ppl_state) #will have something that checks the infected count at certain time intervals
    return ppl_state

            
                
                
        

In [155]:


def add_to_timeline(tic, tim):
    ind=len(tim)-1
    end=0
    while (tic['time']<tim[ind]['time'] and end==0):
        ind=ind-1 
        if(ind<0):
            end=1
    ind=ind+1
    tim.insert(ind, tic)
    return tim
    
#test
tic1=ticket_generator(1,1,1,1)
tic11=ticket_generator(0,1,1,1)
tic111=ticket_generator(6,1,1,1)
tic2=ticket_generator(.5,1,1,1)
tic3=ticket_generator(1.5,1,1,1)
tic5=ticket_generator(5, 1, 1, 1)
tic4=ticket_generator(4.5, 1, 1, 1)
tl=[tic2, tic3, tic4, tic5]
tl2=add_to_timeline(tic111, tl)
print(tl2)


    


[{'time': 0.5, 'owner': 1, 'action': 1, 'level': 1}, {'time': 1.5, 'owner': 1, 'action': 1, 'level': 1}, {'time': 4.5, 'owner': 1, 'action': 1, 'level': 1}, {'time': 5, 'owner': 1, 'action': 1, 'level': 1}, {'time': 6, 'owner': 1, 'action': 1, 'level': 1}]


In [156]:
#start things out
num_ppl=100
ppl_state=[0]*num_ppl 
ppl_state[0]=1 #make a record of everyone's current state, starting with 1 infected person
connect=network_writer(len(ppl_state)) #make a network of connections btwn the ppl -- everyone is connected to everyone rn
tinfinit=nextTime(spreadparam) #generate the time of the first infection
tcureinit=nextTime(cureparam) #generate the time of patient 0's cure
while(tcureinit<tinfinit):
    tcureinit=nextTime(cureparam) #ensure that that time occurs after patient 0 has infected someone -- just so our sim goes somewhere
a=random.choice(connect[0]) #generate patient 0's infection target 
init_inf=ticket_generator(tinfinit, 0, a, 1) #generate a ticket for the first infection 
init_cure=ticket_generator(tcureinit, 0, -1, 1) # generate a ticket for the first cure
timeline=[init_inf, init_cure] #put these 2 tickets on timeline
timeholder=0 #timeholder is the index where we are on the timeline (start at 0)
t=0 #we start at a time of 0 
t_max=500 #we want to go to 1000 time units

#ok time for the while loop
while t<t_max:
    if timeholder>(len(timeline)-1):
        print('disease eradicated')
        break
    nexttic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
    if (nexttic['action']==-1): #if it's a cure ticket, use the corresponding handler
        ppl_state=curetickethandler(nexttic, ppl_state)
    else: #if it's a spread ticket, use the corresponding handler
        vec=spreadtickethandler(nexttic, timeline, spreadparam, cureparam, ppl_state, t)
        timeline=vec[0]
        ppl_state=vec[1]
    t=nexttic['time'] #new time based on the ticket we just read
    timeholder=timeholder+1 #increment timeholder






7 is already infected!
disease eradicated


In [157]:
print(timeline)

[{'time': 14.936417043603376, 'owner': 0, 'action': 7, 'level': 1}, {'time': 20.77253922949687, 'owner': 0, 'action': 7, 'level': 1}, {'time': 23.918006735607968, 'owner': 7, 'action': -1, 'level': 1}, {'time': 25.039979520119886, 'owner': 0, 'action': -1, 'level': 1}]


In [158]:
print(timeholder)

4


In [159]:
print(t)

25.039979520119886


In [160]:
print(sum(ppl_state)/num_ppl)

0.0
